#Predicting the probability of a song beeing chosen by Obama in his playlists

##Data exploration

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [1]:
from google.colab import auth

# Will collect your credentials
auth.authenticate_user()

In [27]:
# Query Bigquery
query = "SELECT * FROM `music-recommendation-system-24.ml_tables_eu.spotify_songs_obama_concat_csv_V0`"
project = "music-recommendation-system-24"

df_full = pd.read_gbq(query=query, project_id=project)

<ipython-input-27-a41486bfeaa1>:5: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_full = pd.read_gbq(query=query, project_id=project)


In [8]:
df_full.head()

,is_in_obama_playlist,track_id,track_name,track_album_name,track_artist,track_popularity,track_album_release_year,genre,duration_min,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1,0UZFhlhUMrYyzzyOHkylM3,Let's Get Lost,Chet Baker Sings And Plays (Remastered 2004),Chet Baker,52,1955,Jazz,3.73,52.0,37.00,C,-9.350,Major,3.00,84.0,0.0,11.00,55.0,78
1,1,3aEJMh1cXKEjgh52claxQp,I've Got You Under My Skin - Remastered 1998,Songs For Swingin' Lovers! (Remastered),Frank Sinatra,66,1956,Jazz,3.73,58.0,25.00,C#,-12.610,Major,4.00,45.0,0.0,11.00,59.0,127
2,0,6qb0O5HW7H4q6R0AbP38Ko,Long Tall Sally (The Thing),Here's Little Richard [Remastered & Expanded],Little Richard,1,1957,rock,2.20,47.4,59.80,F,-5.023,Major,9.73,76.8,0.0,8.41,91.7,184
3,1,6nKeLkmlyK3ooDoDUxZVgP,Lush Life,Ella Fitzgerald Sings The Duke Ellington Song ...,"Ella Fitzgerald,Oscar Peterson",18,1957,Jazz,3.67,40.0,4.38,A#,-19.260,Major,3.91,97.4,0.0,8.95,17.6,91
4,1,4smkJW6uzoHxGReZqqwHS5,I'll Be Seeing You,Billie Holiday,Billie Holiday,67,1957,Jazz,3.53,36.0,3.00,C#,-20.480,Major,4.00,99.0,6.0,12.00,19.0,90


In [9]:
df_full.shape

(33004, 20)

In [10]:
df_full.isna().sum()

,0
is_in_obama_playlist,0
track_id,0
track_name,0
track_album_name,0
track_artist,0
track_popularity,0
track_album_release_year,0
genre,0
duration_min,0
danceability,0


In [14]:
df_full["is_in_obama_playlist"].value_counts()

,count
is_in_obama_playlist,
0,32690
1,314


##Cleaning for ML

In [32]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33004 entries, 0 to 33003
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   is_in_obama_playlist      33004 non-null  Int64  
 1   track_id                  33004 non-null  object 
 2   track_name                33004 non-null  object 
 3   track_album_name          33004 non-null  object 
 4   track_artist              33004 non-null  object 
 5   track_popularity          33004 non-null  Int64  
 6   track_album_release_year  33004 non-null  Int64  
 7   genre                     33004 non-null  object 
 8   duration_min              33004 non-null  float64
 9   danceability              33004 non-null  float64
 10  energy                    33004 non-null  float64
 11  key                       33004 non-null  object 
 12  loudness                  33004 non-null  float64
 13  mode                      33004 non-null  object 
 14  speech

In [28]:
df = df_full.drop(columns=["track_name", "track_album_name"])

In [29]:
df = df.set_index("track_id")

##Modeling

In [31]:
from re import X
from sklearn.model_selection import train_test_split

# every columns in X variable except re_purchase which is our target
X = df_ml.drop(columns=["is_in_obama_playlist"])
y = df_ml["is_in_obama_playlist"]

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# store customers_ids for after
test_customers_ids = X_test.index

###Dealing with categorical features

In [30]:
df.head(2)

,is_in_obama_playlist,track_artist,track_popularity,track_album_release_year,genre,duration_min,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
track_id,,,,,,,,,,,,,,,,,
0UZFhlhUMrYyzzyOHkylM3,1,Chet Baker,52,1955,Jazz,3.73,52.0,37.0,C,-9.35,Major,3.0,84.0,0.0,11.0,55.0,78
3aEJMh1cXKEjgh52claxQp,1,Frank Sinatra,66,1956,Jazz,3.73,58.0,25.0,C#,-12.61,Major,4.0,45.0,0.0,11.0,59.0,127


In [33]:
X_train_cat = X_train[["track_artist", "genre", "key", "mode"]]
X_test_cat = X_test[["track_artist", "genre", "key", "mode"]]

In [38]:
ohe = OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown="ignore")

In [39]:
X_train_cat_encoded = ohe.fit_transform(X_train_cat)
X_test_cat_encoded = ohe.transform(X_test_cat)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [40]:
X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded, columns=ohe.get_feature_names_out())
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded, columns=ohe.get_feature_names_out())

In [41]:
X_train_cat_encoded_df.head()

,track_artist_!!!,track_artist_!deladap,"track_artist_""Dear Evan Hansen"" August 2018 Broadway Cast",track_artist_#TocoParaVos,track_artist_$ANFI,track_artist_$IFRA,track_artist_$uicideBoy$,track_artist_(G)I-DLE,track_artist_(Sandy) Alex G,track_artist_*NSYNC,...,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,mode_Major,mode_Minor,mode_minor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


###Dealing with numerical features

In [42]:
df.head(2)

,is_in_obama_playlist,track_artist,track_popularity,track_album_release_year,genre,duration_min,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
track_id,,,,,,,,,,,,,,,,,
0UZFhlhUMrYyzzyOHkylM3,1,Chet Baker,52,1955,Jazz,3.73,52.0,37.0,C,-9.35,Major,3.0,84.0,0.0,11.0,55.0,78
3aEJMh1cXKEjgh52claxQp,1,Frank Sinatra,66,1956,Jazz,3.73,58.0,25.0,C#,-12.61,Major,4.0,45.0,0.0,11.0,59.0,127


In [43]:
X_train_num = X_train[["track_popularity", "track_album_release_year", "duration_min", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]
X_test_num = X_test[["track_popularity", "track_album_release_year", "duration_min", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]

In [44]:
std = StandardScaler()

In [45]:
X_train_num_scaled = std.fit_transform(X_train_num)
X_test_num_scaled = std.transform(X_test_num)

In [46]:
X_train_num_scaled_df = pd.DataFrame(X_train_num_scaled, columns=X_train_num.columns)
X_test_num_scaled_df = pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns)

###Concatenating numerical and categorical features

In [47]:
X_train_processed = pd.concat([X_train_cat_encoded_df, X_train_num_scaled_df], axis=1)
X_test_processed = pd.concat([X_test_cat_encoded_df, X_test_num_scaled_df], axis=1)

##Fitting our logistic regression model

In [48]:
logreg = LogisticRegression()

In [49]:
logreg.fit(X_train_processed, y_train)

LogisticRegression()

In [50]:
y_pred = logreg.predict(X_test_processed)

In [51]:
logreg_accuracy = accuracy_score(y_test, y_pred)
logreg_accuracy

0.9968186638388123

In [58]:
logreg.score(X_test_processed,y_test)

0.9968186638388123

##Fitting a random forrest model

In [52]:
random_forest = RandomForestClassifier()

In [53]:
random_forest.fit(X_train_processed, y_train)

RandomForestClassifier()

In [54]:
random_forest_y_pred = random_forest.predict(X_test_processed)

In [55]:
random_forest_accuracy = accuracy_score(y_test, random_forest_y_pred)
random_forest_accuracy

0.9946977730646872

##Fitting a Nearest Neighbors Model

In [153]:
NearestNeighbors = KNeighborsClassifier()

In [154]:
NearestNeighbors.fit(X_train_processed, y_train)

KNeighborsClassifier()

In [155]:
NearestNeighbors_y_pred = NearestNeighbors.predict(X_test_processed)

In [156]:
NearestNeighbors_accuracy = accuracy_score(y_test, NearestNeighbors_y_pred)
NearestNeighbors_accuracy

0.9895470383275261

##Predicting Obama's future musical taste with logistic regression

###Creating the song list

In [128]:
# Query Bigquery
query = "SELECT * FROM `music-recommendation-system-24.machine_learning_tables.spotify_top_100_for_prediction_csv_V0`"
project = "music-recommendation-system-24"

df_full_prediction = pd.read_gbq(query=query, project_id=project)

<ipython-input-128-db99fd32f70f>:5: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_full_prediction = pd.read_gbq(query=query, project_id=project)


In [129]:
df_full_prediction.head()

,is_in_30k_songs,is_in_Obama_playlists,track_id,track_name,track_artist,track_popularity,track_album_name,track_album_release_year,duration_min,genres,...,key_name,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0,0,0nrRP2bk19rLc0orkWPQk2,Wake Me Up,Avicii,83,TRUE,2013,4.12,EDM,...,D,-5.697,Major,5.23,0.38,0.12,16.1,64.3,124,4
1,0,0,7BqHUALzNBTanL6OvsqmC1,Happier,"Marshmello,Bastille",82,Happier,2018,3.57,EDM,...,F,-2.749,Major,4.52,19.10,0.00,16.7,67.1,100,4
2,0,0,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),"The Kid LAROI,Justin Bieber",78,STAY (with Justin Bieber),2021,2.36,Pop,...,C#,-5.484,Major,4.83,3.83,0.00,10.3,47.8,170,4
3,0,0,1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,Taylor Swift,89,Lover,2019,2.97,Pop,...,A,-5.707,Major,15.70,11.70,0.00,10.5,56.4,170,4
4,0,0,5wANPM4fQCJwkGd4rN57mH,drivers license,Olivia Rodrigo,82,SOUR,2021,4.03,Pop,...,A#,-8.810,Major,5.78,76.80,0.00,10.6,13.7,144,4


###Cleaning the table

In [130]:
df_full_prediction.loc[df_full_prediction["track_name"] == "Mr. Brightside", "track_album_release_year"] = 2004

In [131]:
df_prediction = df_full_prediction.drop(columns=["track_name", "track_album_name","is_in_Obama_playlists", "is_in_30k_songs"])

In [132]:
df_prediction["key"] = df_prediction["key_name"]
df_prediction = df_prediction.drop(columns=["key_name"])
df_prediction["genre"] = df_prediction["genres"]
df_prediction = df_prediction.drop(columns=["genres"])

###Preparing the data for ML

In [133]:
df_prediction = df_prediction.set_index("track_id")

In [134]:
df_prediction_cat = df_prediction[["track_artist", "genre", "key", "mode"]]
df_prediction_num = df_prediction[["track_popularity", "track_album_release_year", "duration_min", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]

In [135]:
df_prediction_cat_encoded = ohe.transform(df_prediction_cat)
df_prediction_num_scaled = std.transform(df_prediction_num)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [136]:
df_prediction_cat_encoded_df = pd.DataFrame(df_prediction_cat_encoded, columns=ohe.get_feature_names_out())
df_prediction_num_scaled_df = pd.DataFrame(df_prediction_num_scaled, columns=df_prediction_num.columns)

In [137]:
df_prediction_processed = pd.concat([df_prediction_cat_encoded_df, df_prediction_num_scaled_df], axis=1)

###Predicting

In [139]:
logreg_predictions = logreg.predict(df_prediction_processed)
logreg_predictions

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [148]:
proba = pd.DataFrame(logreg.predict_proba(df_prediction_processed), columns=["Obama dislikes", "Obama likes"])
proba

,Obama dislikes,Obama likes
0,0.582499,0.417501
1,0.495628,0.504372
2,0.201297,0.798703
3,0.080553,0.919447
4,0.190289,0.809711
5,0.102582,0.897418
6,0.047378,0.952622
7,0.118105,0.881895
8,0.128458,0.871542
9,0.826947,0.173053


In [149]:
proba["Obama likes"]=round(proba["Obama likes"]*100,2)
proba["Obama dislikes"]=round(proba["Obama dislikes"]*100,2)

proba.head()

,Obama dislikes,Obama likes
0,58.25,41.75
1,49.56,50.44
2,20.13,79.87
3,8.06,91.94
4,19.03,80.97


Merge the probability table in the df_full_preiction table using the index as key.

In [150]:
df_full_prediction_proba = df_full_prediction.merge(proba, left_index=True, right_index=True)

In [151]:
df_full_prediction_proba.head()

,is_in_30k_songs,is_in_Obama_playlists,track_id,track_name,track_artist,track_popularity,track_album_name,track_album_release_year,duration_min,genres,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,Obama dislikes,Obama likes
0,0,0,0nrRP2bk19rLc0orkWPQk2,Wake Me Up,Avicii,83,TRUE,2013,4.12,EDM,...,Major,5.23,0.38,0.12,16.1,64.3,124,4,58.25,41.75
1,0,0,7BqHUALzNBTanL6OvsqmC1,Happier,"Marshmello,Bastille",82,Happier,2018,3.57,EDM,...,Major,4.52,19.10,0.00,16.7,67.1,100,4,49.56,50.44
2,0,0,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),"The Kid LAROI,Justin Bieber",78,STAY (with Justin Bieber),2021,2.36,Pop,...,Major,4.83,3.83,0.00,10.3,47.8,170,4,20.13,79.87
3,0,0,1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,Taylor Swift,89,Lover,2019,2.97,Pop,...,Major,15.70,11.70,0.00,10.5,56.4,170,4,8.06,91.94
4,0,0,5wANPM4fQCJwkGd4rN57mH,drivers license,Olivia Rodrigo,82,SOUR,2021,4.03,Pop,...,Major,5.78,76.80,0.00,10.6,13.7,144,4,19.03,80.97


In [152]:
df_full_prediction_proba = df_full_prediction_proba.sort_values(by="Obama likes", ascending=False)
df_full_prediction_proba = df_full_prediction_proba[["track_name", "track_artist", "Obama likes"]]
df_full_prediction_proba

,track_name,track_artist,Obama likes
34,I Wanna Be Yours,Arctic Monkeys,97.63
12,Heat Waves,Glass Animals,96.64
15,Don't Stop Believin' (2022 Remaster),Journey,96.45
6,Memories,Maroon 5,95.26
16,Smells Like Teen Spirit,Nirvana,95.12
13,Demons,Imagine Dragons,94.37
28,As It Was,Harry Styles,93.59
29,Another Love,Tom Odell,92.39
3,Cruel Summer,Taylor Swift,91.94
5,Without Me,Halsey,89.74
